In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

Image_width,Image_height = 400,400
batch_size=50
val_split=0.01
n_train=25000*(1-val_split)
n_val=25000*val_split

import os
train = pd.DataFrame({"name" :os.listdir("../input/train"), 
                      "path" :pd.Series(os.listdir("../input/train")).apply(lambda x : "../input/train/" + str(x)), 
                      "label" :pd.Series(os.listdir("../input/train")).apply(lambda x : str(x).split(".")[0])})

test = pd.DataFrame({"name" :os.listdir("../input/test"), 
                      "path" :pd.Series(os.listdir("../input/test")).apply(lambda x : "../input/test/" + str(x)), 
                     })
from keras.preprocessing.image import ImageDataGenerator
train_image_gen = ImageDataGenerator(rescale=1/255, horizontal_flip=True, validation_split=val_split)

train_generator = train_image_gen.flow_from_dataframe(dataframe = train, x_col = "path", y_col ="label",
                                                      target_size=(Image_width,Image_height), batch_size=batch_size,seed=19,subset='training',shuffle=True,class_mode='categorical')
val_generator = train_image_gen.flow_from_dataframe(dataframe = train, x_col = "path", y_col ="label",
                                                    target_size=(Image_width,Image_height), batch_size=batch_size,seed=19,subset='validation',shuffle=True,class_mode='categorical')

from keras.optimizers import SGD
from keras.layers import Dense, Dropout
from keras.applications.inception_v3 import InceptionV3
from keras.models import Model
InceptionV3_base_model = InceptionV3(weights='imagenet', include_top=False, pooling = "avg") 

x = InceptionV3_base_model.output
#x_dense = Dense(1024,activation='relu')(x)
final_pred = Dense(2,activation='softmax')(x)
model = Model(inputs=InceptionV3_base_model.input, outputs=final_pred)

from keras.callbacks import ModelCheckpoint
cb_checkpoint = ModelCheckpoint('best.hd5', monitor = 'val_loss', save_best_only = True, )

layer_to_Freeze=172    
for layer in model.layers[:layer_to_Freeze]:
    layer.trainable =False
for layer in model.layers[layer_to_Freeze:]:
    layer.trainable=True

sgd = SGD(lr = 0.01, decay = 1e-6, momentum = 0.9, nesterov = True)
model.compile(optimizer=sgd, loss='categorical_crossentropy',metrics=['accuracy'])

model.fit_generator(train_generator, epochs=3,
                                                steps_per_epoch=n_train//batch_size,
                                                validation_data=val_generator,
                                                validation_steps=n_val//batch_size,
                                                verbose=1,
                                                callbacks=[cb_checkpoint],)

model.load_weights('best.hd5')

test_image_gen = ImageDataGenerator(rescale=1/255)
test_generator = test_image_gen.flow_from_dataframe(dataframe = test, x_col = "path", y_col = None,
                                                    target_size=(Image_width,Image_height), batch_size=50, seed=42, class_mode=None, shuffle=False)

y_pred = model.predict_generator(generator=test_generator, steps=int(np.ceil(len(test)/batch_size))-1, workers = 2, verbose=1)
submission = pd.DataFrame({'id':pd.Series(test_generator.filenames),'label':pd.Series(y_pred.clip(min=0.02, max=0.98)[:,1])})
submission['id'] = submission["id"].apply(lambda x: str(x).split("/")[3].split(".")[0]).astype("int")
submission.to_csv('DogVsCats_submission.csv',index=False)